In [8]:
import torch
import torchvision
from PIL import Image, ImageFilter, ImageOps
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from torchvision.datasets import STL10, ImageFolder
import pandas as pd
from pathlib import Path

In [ ]:
custom_transform = transforms.Compose([ transforms.RandomCrop(224), transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor()])
dataset_class = CustomDatasetWithLabels(
    csv_file =  'train.csv',
    root_dir = 'D:/TUD/TU_Dresden/WiSe_2021/Thesis_FZJ/datasets/data_public_leaderboard_phase/', 
    transform = custom_transform)

In [ ]:
custom_transform = transforms.Compose([ transforms.RandomCrop(224), transforms.Grayscale(num_output_channels=3),transforms.ToTensor()
])
dataset_train = CustomDatasetWithLabels(
    csv_file =  'train.csv',
    root_dir = 'D:/TUD/TU_Dresden/WiSe_2021/Thesis_FZJ/datasets/data_public_leaderboard_phase/train', 
    transform = custom_transform)


In [ ]:
train_loader = DataLoader(dataset=dataset_train, batch_size= 8, shuffle=True, num_workers=0)

In [ ]:
num_epoch = 6
for epoch in range(num_epoch):
    for batch_idx, (x,y) in enumerate(train_loader):
        print('Epoch:', epoch+1, end='')
        print('| Batch index:', batch_idx, end='')
        print('| Batch size:', y.size()[0])

In [3]:
def dataset_with_index(DatasetClass):
    class DatasetWithIndex(DatasetClass):
        def __getitem__(self, index):
            data = super().__getitem__(index)
            return (index, *data)
    return DatasetWithIndex


In [ ]:
train_dir = 'D:\TUD\TU_Dresden\WiSe_2021\Thesis_FZJ\datasets\data_public_leaderboard_phase'
transform = transforms.Compose([ transforms.RandomCrop(224), transforms.Grayscale(num_output_channels=3),transforms.ToTensor()
])

In [69]:
class GaussianBlur:
    def __init__(self, sigma= [0.1, 2.0]):
        """Gaussian blur as a callable object.

        Args:
            sigma (Sequence[float]): range to sample the radius of the gaussian blur filter.
                Defaults to [0.1, 2.0].
        """

        self.sigma = sigma

class Solarization:
    """Solarization as a callable object."""

    def __call__(self, img: Image) -> Image:
        """Applies solarization to an input image.

        Args:
            img (Image): an image in the PIL.Image format.

        Returns:
            Image: a solarized image.
        """

        return ImageOps.solarize(img)


class BaseTransform:
    """Adds callable base class to implement different transformation pipelines."""

    def __call__(self, x: Image) -> torch.Tensor:
        return self.transform(x)

    def __repr__(self) -> str:
        return str(self.transform)

class CustomTransform(BaseTransform):
    def __init__(
        self,
        brightness= 0.,
        contrast= 0.,
        saturation= 0.,
        hue =0.,
        color_jitter_prob: float = 0.8,
        gray_scale_prob: float = 0.2,
        horizontal_flip_prob: float = 0.5,
        gaussian_prob: float = 0.5,
        solarization_prob: float = 0.0,
        min_scale: float = 0.08,
        max_scale: float = 1.0,
        crop_size: int = 224,
        mean= (0.485, 0.456, 0.406),
        std= (0.228, 0.224, 0.225),
    ):

        super().__init__()
        self.transform = transforms.Compose(
            [
                transforms.Grayscale(num_output_channels=3),
                transforms.RandomResizedCrop(
                    crop_size,
                    scale=(min_scale, max_scale),
                    interpolation=transforms.InterpolationMode.BICUBIC,
                ),
                transforms.RandomGrayscale(p=gray_scale_prob),
                transforms.RandomApply([GaussianBlur()], p=gaussian_prob),
                transforms.RandomApply([Solarization()], p=solarization_prob),
                transforms.RandomHorizontalFlip(p=horizontal_flip_prob),
                transforms.ToTensor(),
                transforms.Normalize(mean=mean, std=std),
            ]
        )

In [72]:
dataset_class = ThermalBarrierCoating
transform = CustomTransform
train_dataset = dataset_with_index(dataset_class)(train_dir, transform)

In [73]:
train_loader = DataLoader(dataset=train_dataset, batch_size=200, shuffle=True, num_workers=0)
num_epoch = 2
for epoch in range(num_epoch):
    for batch_idx, (x) in enumerate(train_loader):
        print('Epoch:', epoch+1, end='')
        print('| Batch index:', batch_idx, end='')
        print('|',len(x))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class '__main__.CustomTransform'>

In [20]:

class ThermalBarrierCoating(Dataset):
    def __init__(self, train_dir, transform=None):
        self.data_csv = pd.read_csv(train_dir / Path('train.csv')) 
        self.label= self.data_csv['Lifetime']
        self.image_ID = self.data_csv['Image_ID']
        self.transform = transform


    def __getitem__(self, index):
        y_label = self.label[index]
        image = Image.open(os.path.join(train_dir, 'train',self.image_ID[index]) +'.tif')
        if self.transform is not None:
            image = self.transform(image)
        return image, -1

    def __len__(self):
        return self.data_csv['Lifetime'].shape[0]

In [ ]:
diri = train_dir/csv

In [ ]:
diri